In [12]:
#All imports at the top
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [13]:
#Adapter Layer - All functions which interact with s3 storage
#Read csv files to convert to df
    #Remove part of for loop into a function
    #Think of changing parts of the code to be as input arguments, creating default values if needed 
    #Adding what needs to be added in or used like a bucket, key, the decoding and sep, changing the code
def read_csv_to_df(bucket,key,decoding='utf-8',sep=','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

#Write df to s3
    #Renaming the specific name to be more general from bucket_target to bucket, and df_all to df
    #Add a return of true
def write_df_to_s3(bucket,df,key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer,index=False)
    bucket.put_object(Body=out_buffer.getvalue(),Key=key)
    return True

def write_df_to_s3_csv(bucket,df,key):
    out_buffer = StringIO()
    df.to_csv(out_buffer,index=False)
    bucket.put_object(Body=out_buffer.getvalue(),Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files


In [14]:
# Application Layer (core) - ETL
def extract(bucket,date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket,date)]
    df = pd.concat([read_csv_to_df(bucket,obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df,columns,arg_date):
    df = df.loc[:,columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN','Date'],as_index=False).agg(opening_price_eur=('opening_price','min'),closing_price_eur=('closing_price','min'), minimum_price_eur=('MinPrice','min'), maximum_price_eur=('MaxPrice','max'), daily_traded_volume=('TradedVolume','sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'],inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date>=arg_date]
    return df

def load(bucket,df,trg_key,trg_format,meta_key,extract_date_list):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format  
    write_df_to_s3(bucket,df,key)
    update_meta_file(bucket,meta_key,extract_date_list)
    return True

    #Note there are two buckets in the original function so we have to differentiate by name
def etl_report1(src_bucket,trg_bucket,date_list,columns,arg_date,trg_key,trg_format,meta_key):
    df = extract(src_bucket,date_list)
    df = transform_report1(df,columns,arg_date)
    extract_date_list = [date for date in date_list if date >= arg_date]
    load(trg_bucket,df,trg_key,trg_format,meta_key,extract_date_list)
    return True

In [15]:
# Application Layer - not core

# def return_objects(bucket,arg_date,src_format):
#     min_date = datetime.strptime(arg_date,src_format).date() - timedelta(days=1)
#     objects = [obj.key for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= min_date]
#     return objects

#Rewrite the function from above
def return_date_list(bucket,arg_date,src_format, meta_key):
    min_date = datetime.strptime(arg_date,src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0,(today-min_date).days + 1)]    
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:    
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = (min_date + timedelta(days=1)).strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200,1,1).date()
    except bucket.session.client('s3').exceptions.NoSuchKey:
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0,(today-min_date).days + 1)]    
        return_min_date = arg_date
    return return_min_date, return_dates

def update_meta_file(bucket,meta_key,extract_date_list):
    df_new = pd.DataFrame(columns=['source_date','datetime_of_processing'])
    df_new['source_date'] = extract_date_list
    df_new['datetime_of_processing'] = datetime.today().strftime('%Y-%m-%d')
    df_old = read_csv_to_df(bucket, meta_key)
    df_all = pd.concat([df_old,df_new])
    write_df_to_s3_csv(bucket,df_all,meta_key)

In [16]:
#main function entry point
def main():
    #Parameters/configurations
    #Later read config
    arg_date = '2021-05-19'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    trg_bucket = 'xetra-data-etl-destination'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
           'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    meta_key = 'meta_file.csv'
    
    #Init
    s3 = boto3.resource('s3')
    #Note to keep the names different
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    #run application
    extract_date, date_list = return_date_list(bucket_trg, arg_date, src_format, meta_key)
    etl_report1(bucket_src,bucket_trg,date_list,columns,extract_date,trg_key,trg_format,meta_key)

In [17]:
#run
main()

### Reading the uploaded file

In [18]:
trg_bucket = 'xetra-data-etl-destination'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
    
for obj in bucket_trg.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_20230422_151559.parquet
xetra_daily_report_20230422_171304.parquet
xetra_daily_report_20230422_211652.parquet
xetra_daily_report_20230422_220232.parquet
xetra_daily_report_20230423_170319.parquet
xetra_daily_report_20230423_174716.parquet


In [19]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20230423_170319.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [20]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
...,...,...,...,...,...,...,...,...
1038839,XS2437455608,2022-04-19,23.90,24.09,23.90,24.51,5225,0.02
1038840,XS2437455608,2022-04-20,24.13,23.91,23.59,24.13,0,-0.76
1038841,XS2437455608,2022-04-21,23.75,23.61,23.18,23.75,0,-1.25
1038842,XS2437455608,2022-04-22,24.11,25.19,24.00,25.19,1200,6.71
